In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import random
import pandas as pd
import cv2
import os
from skimage import io, util
from skimage.transform import resize


from sklearn.metrics import (
    accuracy_score,
    recall_score,
    precision_score,
    average_precision_score,
)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

from utils.detection import detect_ecocup

On va tester un classifieur choisi arbitrairement pour tester l'algorithme de fenetre glissante


In [2]:
# Lire toutes les images dans le dossier
def read_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            img = io.imread(img_path)
            if img is not None:
                images.append(img)
    return images


pos = read_images_from_folder("local_data/4_normalized_patches/pos")
neg = read_images_from_folder("local_data/4_normalized_patches/neg")

In [3]:
y = [1 for _ in range(len(pos))] + [0 for _ in range(len(neg))]
X = pos + neg

# Suffle les données
indices = np.arange(len(X))
np.random.shuffle(indices)
X_shuffled = np.array(X)[indices]
y_shuffled = np.array(y)[indices]

# creer un jeu d'entrainement et un jeu de test
split_index = int(0.8 * len(X_shuffled))
X_train = X_shuffled[:split_index]
y_train = y_shuffled[:split_index]
X_test = X_shuffled[split_index:]
y_test = y_shuffled[split_index:]

# flatten
X_train_flat = np.array([img.flatten() for img in X_train])
X_test_flat = np.array([img.flatten() for img in X_test])

In [ ]:
clf = RandomForestClassifier(
    n_estimators=100,
)
clf.fit(X_train_flat, y_train)

In [ ]:
# Tester le classifieur
y_pred = clf.predict(X_test_flat)
# Calculer les métriques
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
average_precision = average_precision_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"Recall: {recall:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Average Precision: {average_precision:.2f}")

Accuracy: 0.91
Recall: 0.46
Precision: 0.98
Average Precision: 0.55


In [ ]:
img = io.imread("data/train/images/pos/0000.jpg")
labels = pd.read_csv("data/train/labels_csv/0000.csv")

In [ ]:
detections = detect_ecocup(
    img,
    clf,
)

print(detections)
print(labels)

Calcul de l'IoU entre les fenetres :  [((0, 0), (20, 20), 0.52), ((0, 20), (20, 40), 0.52)] [((0, 0), (20, 20), 0.52), ((0, 40), (20, 60), 0.52)]


TypeError: unsupported operand type(s) for -: 'tuple' and 'tuple'

In [ ]:
from utils.detection import sliding_window

parts, winds = sliding_window(img, 20, 20, 5, 5)

In [ ]:
from utils.detection import get_iou

iou = get_iou(winds[0], winds[1])

Calcul de l'IoU entre les fenetres :  ((0, 0), (20, 20)) ((0, 5), (20, 25))


In [ ]:
winds[0]

((0, 0), (20, 20))

In [ ]:
winds2 = []
for wind in winds:
    ulc = wind[0]
    lrc = wind[1]
    winds2.append((ulc, lrc, 0.4))